## Investment Tool Introduction:

This code/model has successfully created an investment tool that is based soley on technical analysis, but should be used in conjunction with fundamental practices that drive investment decisions. This model provides several different types of information to the user of the tool, of which can be used to predict whether or not a stock will increase by 1% - 5%, the day following a stock's earnings report. 

For example, assume that stock x releases their earnings report after market close today. By this time, the model would be able to give a prediction on whether stock x will increase by 1% to 5% by the end of the following trading day. This would give the investor enough time to purchase the stock the following morning, and then sell before the market closes to realize the 1% to 5% gain. 

The underlying algothrim used to perform predictions was the Naive Bayes Classifier. What this classifier helps determine is the probability that a certain stock x will increase by 1% to 5%, given that certain events have already occured. To be more specific, the model answers the following question:

"What is the probability that the price of stock x will increase by 1% to 5%, by the time the market closes on the trading day following the reported earnings date, given that events x1 through x5 occur:"


- x1 = Reported earnings per share (EPS) were higher than the previous reported date 


- x2 = Reported EPS was higher than the estimated EPS (i.e. the EPS surprise was positive)


- x3 = Adjusted stock price is larger than the previous reported date


- x4 = Intraday return of the stock x, on reported date, is greater than or equal to its median intraday return of the last 5 trading days


- x5 = Intraday return of the SPY etf is on reported date, is greater than or equal to its median intraday return of the last 5 trading days



## Explaining Events


x_1:

The first event could be telling us that the market thinks that the company is going to perform well in the upcoming earnings report, and it reflects that through the price. The adjusted price was used because it takes into account any stock-splits that may have occured. However, the first event could also be a signal that the market is overvaluing the future performance of the company, potentially making it a relatively bad time to buy the stock. 

x_2:

The second event informs the investor that that growth is currently in place or that the company is becoming more efficient by decreasing their cost of goods/expenses. Both could lead to an increase interest in the company and drive the stock price up based on demand.   

x_3:

The third event gives us a better understanding of how incorrect the market was at estimating the performance of a company. The higher the percentage error, also known as the surprise percentage, the better, because it signals under-valuation. During these periods, the investor has the opportunity to purchase a stock for a relatively "low" price and then sell at a 'fair' market price.

x_4:

The fourth event shows that the market has a positive sentiment regarding earnings. It is common to see a stock's intraday return fluctulate throughout the week and it is possible that one of those days, the market overvalued the stock and increased the average intraday return. Thus, the median was used to better reflect the distribution of the most recent intraday returns. If the stock increased higher than its weekly median return, it could signal that the stock has the ability to increase.

x_5:

The fifth event informs the investor on how the overall market is performing in the span of one week. The S&P 500 is index that tracks slightly over 500 publicly traded companies in the United States. This index is constantly used as a benchmark to gage how the overall market is performing. Moreover, this index is tracked by an exchange-traded-fund (ETF) called SPY that was used in this analysis. If the return on SPY, at the end of the reported date, was higher than its weekly median intraday return , it could potentially signal that the market has a positive outlook.


## Model Breakdown

The model provides the following information to the user:

1. Upcoming Earnings Calendar (3 month horizon):

    - Dataframe of companies that are reporting earnings on a given date within the next three months


2. Historical Earnings Report (Individual Company):

    - Dataframe of all the reported EPS instances along with their estimated EPS, surprise EPS, and corresponding dates
    
    
3. Daily Adjusted Prices Report (Individual Company):

    - Dataframe of the 5,000 most recent trading dates that contain intraday data
    
    
4. Naive Bayes Prediciton Model:

    - Current Date provides a buying signal and Future Date provides a model accuracy score for a specific stock:


        - Individual Analysis (Current Date/ Future Date)
        - Multi-Company Analysis (Current Date/ Future Date)


## Gathering Financial Data

The majority of the financial data will be pulled from alphavantage's financial APIs as well as Finhub's financial API. In order to conduct a naive bayes analysis, the user will need a historical earnings report that contains at least 3 years (12 quarters) worth of reported earnings history. This dataframe will yield the first 2 events (x1 and x2).

Moreover, the user will need a record of all the daily adjusted prices (most recent 5,000 trading dates) for a specific stock so that it can be matched up with the dates that earnings were reported. The user will also need a record of all the daily adjusted prices (most recent 5,000 trading dates) for the SPY ETF. 

Finally, the Finhub historical earnings function was used to get the most recent reported earnings information to make buying predictions on the current date (day of reported earnings).

The majority of this model consists of cleaning the dataframes, deriving variables from them using statistics, and merging them properly based on conditions. In addition, given that the alphavantage API has a 5 call per minute limit, the model needed to efficiently call the API so that calls were wasted nor limits exceeded. 

## Model Creation: Fundamental Functions

###  1. Creating the finhubEarningsCalendar Function

This function can only return historical earnings data.

In [1]:
#this function pulls earnings data from finhub and then cleans it so that it can be used in the analysis
#this function only works for past data and does not give upcoming earnings, that is why the alphaVantage api was used in conjuction
#earnings calendar should be different than the alpha because it is more up to date and gives BMO o AMC datapoints
#inputs include:
#date range
#symbol - if no symbol is given, api will return all the stocks that are releasing earnings on that date

def finhubEarningsCalendar(finhub_api,from_date,to_date,symbol):
    
    
    #gathering the earnings calendar data:
    
    #import the library
    import finnhub
      
    #create an instance of the finhub library that contains API key
    finnhub_client = finnhub.Client(api_key=finhub_api)
    
    #get the stock earnings data that from the specified date range
    #this returns a nested dictionary that needs to be pulled apart 
    finub_earn_calendar = finnhub_client.earnings_calendar(_from=from_date, to=to_date, symbol=symbol, international=False)
    
    #the earningsCalendar key contains the values neede for the analysis
    finub_earn_calendar = finub_earn_calendar['earningsCalendar']
    
    #import pandas library
    import pandas as pd
    
    #convert the dictionary into a readable dataframe
    earningsCalendar = pd.DataFrame(data=finub_earn_calendar)
    
    
    #editing the dataframe so that it can be compatible with the alphaVantage dataframes and to use in the model:
    
    #filter out the following columns: quarter, revenueActual, revenueEstimate, and year:
    earningsCalendar.loc[:,['date', 'epsActual','epsEstimate','hour','symbol']]
    
    #change the column names:
    earningsCalendar.rename(columns = {'epsEstimate':'estimatedEPS'},inplace=True)
    earningsCalendar.rename(columns = {'date':'reportedDate'},inplace=True)
    earningsCalendar.rename(columns = {'epsActual':'reportedEPS'},inplace=True)
    
    #change the positions of the columns:
    #list of the column names
    column_names = ["symbol", "reportedDate", "hour","reportedEPS","estimatedEPS"]
    
    #reoder the dataframe
    earningsCalendar = earningsCalendar.reindex(columns=column_names)  
    
    #filter out the nones, of which can be found in the following columns:
    earningsCalendar =  earningsCalendar.loc[(earningsCalendar['estimatedEPS'].notnull())]
    earningsCalendar =  earningsCalendar.loc[(earningsCalendar['reportedEPS'].notnull())]
    
    
    #add the surprise amount column
    earningsCalendar['surprise'] = earningsCalendar['reportedEPS'] - earningsCalendar['estimatedEPS'] 

    #add the surprise perecentage column
    earningsCalendar['surprisePercentage'] = round(((earningsCalendar['reportedEPS'] /earningsCalendar['estimatedEPS'])-1)*100,4)
    
    
    #return the dataframe
    return earningsCalendar

### 2. Creating the alphaDataRetriever Function

In [2]:
#create a function that:
#individual earnings table for a specific stock
#daily adjusted prices for a specific stock:
#it also calculates the Intraday_Return and Weekly_Median_Return to be used later in the analysis section

#inputs include:
#type of data to retrieve where:
#1 = historical earnings
#2 = daily ajusted prices
#api key for alphaVantage


def alphaDataRetriever(api_key, data_of_interest, symbol,date_of_interest):
    #import necessary libraries
    import pandas as pd
    import csv
    import requests
    
    
    #remember that alphavantage only takes 5 calls per mintute and 500 throughout the entire day
    #for the earnings calendar call, symbol is not important

    if data_of_interest == '1':

        CSV_URL = 'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&horizon=3month&apikey=' + api_key

        with requests.Session() as s:

            download = s.get(CSV_URL)
            decoded_content = download.content.decode('utf-8')
            cr = csv.reader(decoded_content.splitlines(), delimiter=',')
            earnings_calendar = pd.DataFrame(data =cr)

        #edit the dataframe:
        #make the first row the column names
        earnings_calendar = earnings_calendar.rename(columns=earnings_calendar.iloc[0])

        #delete the first row since it has been used already to name the columns
        earnings_calendar = earnings_calendar.iloc[1:,:]

        #filter by the date of interest and loc the dataframe
        ec = earnings_calendar.loc[earnings_calendar.reportDate == date_of_interest]

        #not all stocks have an EPS estimate, these should be excluded from the dataframe
        #dataframe = dataframe[dataframe.column then the condition]
        ec = ec[ec.estimate != '']

        #not all stocks are traded in USD, but this project will only focus on the USD currency
        ec = ec[ec.currency == 'USD']

        #remove the irrelevant columns
        ec = ec.drop(['fiscalDateEnding','currency'],1)

        #display dataframe
        return ec


    
    elif data_of_interest == '2':

        #format of how the url should be structured before using the api
        earnings_history_url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol=' + symbol +'&apikey=' + api_key

        #makes the request using the formatted url
        r = requests.get(earnings_history_url)
        data = r.json()

        #the data variable stores the data in json structure and the value of key = quarterlyEarnings is the point of interest
        quarterly_earnings = data['quarterlyEarnings']

        #convert this json into a dataframe
        quarterly_earnings = pd.DataFrame(quarterly_earnings)
        
        quarterly_earnings = quarterly_earnings.drop('fiscalDateEnding',1)
        
        return quarterly_earnings
    
    #this api call will return the most recent data avaialble
    
    elif data_of_interest == '3':
        #getting the time series data
        #this url contains the format needed to make the api call for time series data
        daily_prices_url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=' + symbol + '&outputsize=full' +'&apikey=' + api_key

        #makes the api request/call
        daily_price_request = requests.get(daily_prices_url)

        #stores the values into a dictionary
        daily_prices = daily_price_request.json()
        
        #stores the relevant information into a dictionary
        dictionary_of_dp = daily_prices['Time Series (Daily)']

        #convert this json into a dataframe, where the keys are the rows and the values are the columns 
        historical_prices = pd.DataFrame.from_dict(dictionary_of_dp, orient = 'index')
        
        #turn column values into numeric values
        historical_prices = historical_prices.apply(pd.to_numeric)
    
        #perform return calculations for each specific date
        historical_prices['Intraday_Returns'] = round((historical_prices['5. adjusted close']/historical_prices['1. open']-1)*100,4)

        #find the previous 5 day median intraday return and create the column 
        #the value will take into account the previous 5 trading days
        historical_prices['Weekly_Median_Return'] = historical_prices['Intraday_Returns'].rolling(window=5).median().shift(-5)

        #the indexes are the dates thus make that a column labeled reportedDate because that will be the key column for the inner join
        historical_prices['trading_date'] = historical_prices.index
        
        #reset the index since its a column now
        historical_prices.reset_index(drop=True, inplace=True)
        
        #remember that the model could break if there is not enough historical data
        return historical_prices
    
    #tell the analyst that their value is not valid
    else:
        print('Wrong Input for the alphaDataRetriever - Try again.')

### 3. Creating the historicalEarningsTransformer Function


In [3]:
#the model needs to understand what date the user is inputting
def historicalEarningsTransformer(date_type,historical_earnings,finhub_function):
    
    #assuming that the date provided by the analyst is after market close on the reported date:

    #import necessary libraries
    import pandas as pd

    #don't alter the dataframes that were used as inputs, instead, make a copy of each:
    
    #this variable holds a stock's individual earnings history
    earnings_hist = historical_earnings.copy(deep=True)
    

    #for training purposes,only keep the rows that are relevant.
    #the api feeds in the most current earnings and as a result, the historical prices will not line up
    #let those rows be used for the testing portion, while the remaining events get analyzed
    #no need for rows that don't contain information such as none. If none is found in the estimate, then the remaining 2 columns will be none
   
    
    #if analyst chooses the reporting date of today, after market close: 
    
    if date_type == '1':
        
        #keep the first cell in the first row because it contains the date of the most recent/upcoming earnings event
        #this date will be used to gather several variables such as intraday event, SPY event, Adjusted Price event
        report_date_of_interest = earnings_hist.iloc[0,0]

        
        #create common column name so that the dataframes can be merged properly 
        #common name is equal to 'column_of_interest'
        earnings_hist.rename(columns = {'reportedDate':'column_of_interest'}, inplace = True)

        #filter out the nones, of which can be found in the following columns:

        #the first row will always be none because it is a future date or the current report date
        #this api does not update in real time
        earnings_hist =  earnings_hist.loc[(earnings_hist['reportedEPS']!= 'None')]
        earnings_hist =  earnings_hist.loc[(earnings_hist['estimatedEPS']!= 'None')]
        earnings_hist =  earnings_hist.loc[(earnings_hist['surprise']!= 'None')]
        earnings_hist =  earnings_hist.loc[(earnings_hist['surprisePercentage']!= 'None')]
        
        
        #drop the columns that don't align with the earnings_hist dataframe. This is needed to merge properly
        finhub_function = finhub_function.drop('hour',1)
        finhub_function = finhub_function.drop('symbol',1)
        
        #create common column name
        finhub_function.rename(columns = {'reportedDate':'column_of_interest'}, inplace = True)
        
        
        #concat the dataframes so that the surprise_event and earnings_event x variables can be calculated 
        earnings_hist = pd.concat([finhub_function,earnings_hist])
        
        #convert the following columns to numeric in order to do math operations
        earnings_hist["reportedEPS"] = pd.to_numeric(earnings_hist["reportedEPS"])
        earnings_hist["estimatedEPS"] = pd.to_numeric(earnings_hist["estimatedEPS"])
        earnings_hist["surprisePercentage"] = pd.to_numeric(earnings_hist["surprisePercentage"])


        #add the column that determines if current earnings were greater than last time
        earnings_hist['earnings_event'] = (earnings_hist['reportedEPS'] > earnings_hist['reportedEPS'].shift(-1)).astype(int)

        #add the column that determines if the surprise variable is positive or not
        earnings_hist['surprise_event'] = (earnings_hist['surprisePercentage'] > 0).astype(int)
        
        return earnings_hist

     
    elif date_type == '2':

        #create common column name so that the dataframes can be merged properly 
        #common name is equal to 'column_of_interest'
        earnings_hist.rename(columns = {'reportedDate':'column_of_interest'}, inplace = True)

        #filter out the nones, of which can be found in the following columns:

        #the first row will always be none because it is a future date or the current report date
        #this api does not update in real time
        earnings_hist =  earnings_hist.loc[(earnings_hist['reportedEPS']!= 'None')]
        earnings_hist =  earnings_hist.loc[(earnings_hist['estimatedEPS']!= 'None')]
        earnings_hist =  earnings_hist.loc[(earnings_hist['surprise']!= 'None')]
        earnings_hist =  earnings_hist.loc[(earnings_hist['surprisePercentage']!= 'None')]
        
        #convert the following columns to numeric in order to do math operations
        earnings_hist["reportedEPS"] = pd.to_numeric(earnings_hist["reportedEPS"])
        earnings_hist["estimatedEPS"] = pd.to_numeric(earnings_hist["estimatedEPS"])
        earnings_hist["surprisePercentage"] = pd.to_numeric(earnings_hist["surprisePercentage"])


        #add the column that determines if current earnings were greater than last time
        earnings_hist['earnings_event'] = (earnings_hist['reportedEPS'] > earnings_hist['reportedEPS'].shift(-1)).astype(int)

        #add the column that determines if the surprise variable is positive or not
        earnings_hist['surprise_event'] = (earnings_hist['surprisePercentage'] > 0).astype(int)
        
        return earnings_hist


    else:
        print('Invalid input. Please Try Again')



### 4. Creating the dailyAdjustedPriceTransformer Function

In [4]:
#this function takes in the daily adjusted dataframe, individual earnings dataframe, and the spy dataframe 
#it calculates 2 x variables and returns a new merged dataframe that will be merged one final time in the next function

def dailyAdustedPriceTransformer(hp,individual_earnings,spy):
    
    #editing dataframes via renaming columns and removing unnessary columns:

    #before joining the 3 dataframes, there needs to be a common column name
    #change the name of the date columns to column_of_interest
    
    historical_prices = hp.copy(deep=True)
    
    #change the name
    historical_prices.rename(columns = {'trading_date':'column_of_interest'}, inplace = True)


    #historical_prices dataframe editting:
    #in historical_prices, change the column name of '5. adjusted close' to 'stock_adjusted_close'
    historical_prices.rename(columns = {'5. adjusted close':'stock_adjusted_close'}, inplace = True)

    #in historical_prices, change the column name of 'Intraday_Returns' to 'stock_intraday_returns'
    historical_prices.rename(columns = {'Intraday_Returns':'stock_intraday_returns'}, inplace = True)

    #in historical_prices, change the column name of 'Weekly_Median_Return' to 'stock_weekly_median_return'
    historical_prices.rename(columns = {'Weekly_Median_Return':'stock_weekly_median_return'}, inplace = True)
    
    #code below is used to calculate the intraday_event and adjusted_price_event variables:
    
    
    #use the 'column_of_interest' of the individual earnings dataframe 
    reported_dates_list = individual_earnings['column_of_interest'].tolist()
    
    #this line filters out all the dates that are not a previous reporting date
    historical_prices = historical_prices.loc[historical_prices['column_of_interest'].isin(reported_dates_list)]

    #add the column that determines if the intraday return of the earnings release date is larger than that of the median
    historical_prices['intraday_event'] = (historical_prices['stock_intraday_returns'] > historical_prices['stock_weekly_median_return']).astype(int)

    #add the column that determines if the adjusted share price is larger than that of last earnings call
    historical_prices['adjusted_price_event'] = (historical_prices['stock_adjusted_close'] > historical_prices['stock_adjusted_close'].shift(-1)).astype(int)


    #spy dataframe editting:

    #change name
    spy.rename(columns = {'trading_date':'column_of_interest'}, inplace = True)
    #in spy, change the column name '5. adjusted close' to 'stock_adjusted_close'
    spy.rename(columns = {'5. adjusted close':'spy_adjusted_close'}, inplace = True)

    #in spy, change the column name 'Intraday_Returns' to 'stock_adjusted_close'
    spy.rename(columns = {'Intraday_Returns':'spy_intraday_returns'}, inplace = True)

    #in historical_prices, change the column name of 'Weekly_Median_Return' to 'spy_weekly_median_return'
    spy.rename(columns = {'Weekly_Median_Return':'spy_weekly_median_return'}, inplace = True)
    
    #add the column that determines if the intraday return of the earnings release date is larger than that of the median
    spy['spy_event'] = (spy['spy_intraday_returns'] > spy['spy_weekly_median_return']).astype(int)
    
    #add the spy intraday so that the analyst can see how the variables were created
    historical_prices['spy_intraday_returns'] = spy['spy_intraday_returns']
    
    #add the spy median so that the analyst can see how the variables were created
    historical_prices['spy_weekly_median_return'] = spy['spy_weekly_median_return']
        
    #add the spy median so that the analyst can see how the variables were created
    historical_prices['spy_adjusted_close'] = spy['spy_adjusted_close']
        
    #now put the values of spy dataframe into the  hist prices
    historical_prices['spy_event'] = spy['spy_event']
    
    
    #display the new dataframe with the 3 variables
    return(historical_prices)


### 5. Creating the finalTransformation Function

In [76]:
#merges the dataframes and returns the clean variables

def finalTransformation(individual_hist_earn,both_dailyAdjustedPrices,daily_prices,output_type,date_type):
    
    #merge cells
    inner_merged_master = individual_hist_earn.merge(both_dailyAdjustedPrices,on='column_of_interest')
    print()
    print('inner merged function')
    display(inner_merged_master)
    #add the class variable:

    #this variable holds all the daily adjusted values for a particular stock, not just values that pertain to date of interest
    master_daily_adjusted = daily_prices.copy(deep=True)
        
    #change the name
    master_daily_adjusted.rename(columns={'trading_date':'column_of_interest'},inplace=True)

    #create list to reference
    reported_dates_list = inner_merged_master['column_of_interest'].tolist()
    
    #master_hist is the dataframe that only includes the dates that match up with the dates that earnings are reported
    master_hist = master_daily_adjusted.loc[master_daily_adjusted['column_of_interest'].isin(reported_dates_list)]
    
    
    #if the current date is the reported earnings date then remove the first row of the master_hist because it will appear there
    if date_type == '1':
        #remove the first row
        master_hist = master_hist.iloc[1:,:]
    
    else:
        master_hist = master_hist
        
    
    
    #notice how the dates in the column_of_interest are one day after the reported earnings date
    #store the index values from the list. This will tell us the index where the earnings reported date is located at
    row_index_list = master_hist.index.tolist()


    #alter the values of the elements in the list so that element = element -1
    #this will give us the value of the index pertaining to returns one trading day following reported release
    #the way the dataframe is filtered is by most recent date to the most future date

    for i in range( len(row_index_list)):
        row_index_list[i] = row_index_list[i] - 1


    #lock the dataframe so that only the row indexes from the list come up
    master_daily_adjusted = master_daily_adjusted.loc[row_index_list,:]

    #display the master_daily_adjusted 
#     print('This dataframe contains the price information associated with the date that is one trading day after the reported earnings date:\n')
#     display(master_daily_adjusted.head(15))

    #rename the column so that the analyst knows what this value is. It's the price of the stock one full trading day after earnings have been released
    master_daily_adjusted.rename(columns = {'5. adjusted close':'after_report_adjusted_price'},inplace=True)
    
    print()
    print('master daily adjsuted')
    display(master_daily_adjusted)
    
    #up to this point, the after_report_adjusted_price column exists
    #store the values of the after_report_adjusted_price in a list to 
    reported_adjusted_close_prices = master_daily_adjusted['after_report_adjusted_price'].tolist()
    
    if date_type == '1':
        
        #since the first row was deleted, a value needs to be inserted to keep the lengths of the arrays the same
        #put the value zero in the first position of the list
        reported_adjusted_close_prices.insert(0,0)

    else:
        reported_adjusted_close_prices = reported_adjusted_close_prices



    #add the values from the list to a new column in the inner master df
    inner_merged_master['after_report_adjusted_price'] = reported_adjusted_close_prices

    #calculate the adjusted price in a new column
    inner_merged_master['return_pct_difference'] = round((inner_merged_master['after_report_adjusted_price']/inner_merged_master['stock_adjusted_close']-1)*100,2)

    #calculate the class variable
    inner_merged_master['class_variable'] = ((inner_merged_master['return_pct_difference'] >= 1) & (inner_merged_master['return_pct_difference'] <= 5) ).astype(int)
    
    #display inner_merged
#     print('This dataframe contains all the variables associated with building the model:\n')
#     display(inner_merged_master)

    #keep relevant columns:
    
    #relevant columns
    model_variables = ['earnings_event','surprise_event','adjusted_price_event','intraday_event','spy_event','class_variable']

    #filter the dataframe
    training_data = inner_merged_master[model_variables]        

    
    #this is the logical display of how the variables were created
    if output_type  == '1':
         
        #display the following dataframe
        #relevant columns
        relevantColumns = ['column_of_interest','reportedEPS','estimatedEPS','surprise','surprisePercentage','stock_adjusted_close','stock_intraday_returns','stock_weekly_median_return','spy_adjusted_close','spy_intraday_returns','spy_weekly_median_return','after_report_adjusted_price','return_pct_difference','earnings_event','surprise_event','adjusted_price_event','intraday_event','spy_event','class_variable']
        
        #filter the dataframe
        logical_dataframe = inner_merged_master[relevantColumns]
        
        #display
        print('This is the dataframe with the columns that were used to construct the variables of the model:\n')
        display(logical_dataframe)

            
    #this returns the clean variables, ready to be analyzed
    elif output_type  == '2':
        print('Clean Variables:\n')
        
        

    else:
        print('Invalid Input. Please try again.')
   

    return training_data

### 6. Creating the naiveBayesPredictor Function

In [6]:
#create the function to peform the analysis
#most import input of this function are the clean variables created in the variableTransformer function that holds the clean variables
#this holds the dataframe that will be broken down into train and test data

def naiveBayesPredictor(cleanVariables,dateType):
    
    #import the tools
    from sklearn.naive_bayes import BernoulliNB

    #start by breaking down the data into their respective sections
    #at this point, the variableTransformer made sure there was enough data to conduct a proper analysis
    
    #break the data into x and y variables
    #for training purposes the test data will consist of full year prediction
    #the dataframe has the most recent quarters at the beginning, thus use the first 4 rows as the testing data
    
    #if the current date is the earnings report date
    if dateType == '1':
            
        #the train variables will be the actual results of one year prior
        x_train_variables = cleanVariables.iloc[5:,0:5]

        y_train_variables = cleanVariables.iloc[5:,5]

        #the first row contains the data for the prediction that is going to be used for the buy or not buy signal
        x_prediction_set = cleanVariables.iloc[0:1,0:5]
        
        #test the model on a full year worth of data (most recent year)
        x_test_variables = cleanVariables.iloc[1:5,0:5]
        y_test_variables = cleanVariables.iloc[1:5,5]
    
    #if the current date is not the same as earnings report date  
    elif dateType == '2':
        
        #the train variables will be the actual results of one year prior
        x_train_variables = cleanVariables.iloc[4:,0:5]
        y_train_variables = cleanVariables.iloc[4:,5]

        #test the model on a full year worth of data (most recent year)
        x_test_variables = cleanVariables.iloc[0:4,0:5]
        y_test_variables = cleanVariables.iloc[0:4,5]
    
    #create an instance of the model
    naive_bayes_model = BernoulliNB(binarize=0.0)
    
    #fit the training variables to the model model
    naive_bayes_model.fit(x_train_variables, y_train_variables)
    
    #make predictions with the model
    predictions = naive_bayes_model.predict(x_test_variables)
    
    #import the metric tools
    from sklearn.metrics import multilabel_confusion_matrix
    from sklearn.metrics import classification_report
    
    #create an instance where the confusion matrix will live
    multi_cm = multilabel_confusion_matrix(y_test_variables, predictions)

    #first row
    tp = multi_cm[0][0][0]
    fp = multi_cm[0][0][1]

    #second row
    fn = multi_cm[0][1][0]
    tn = multi_cm[0][1][1]

    #get the relevant metrics from the classification report
    cr = classification_report(y_test_variables, predictions,output_dict=True)
    precision = cr['weighted avg']['precision'] 
    recall = cr['weighted avg']['recall'] 
    f1_score = cr['weighted avg']['f1-score'] 

    #score the model
    model_score = round(naive_bayes_model.score(x_test_variables,y_test_variables)*100,2)
        
    #both models need to be evaluated for their performance:
    
    #create the point at which one could deem this model accurate
    bench_mark_precision_WA = 0.75

    #if the models precisionWA is greater than or equal to the bench_mark_precision_WA then recommend the analyst use the model
    if precision >= bench_mark_precision_WA:
        model_recommendation = 'Valid'

    #if precision wa is less than the benchmark, the model should NOT be used to make any buying decisions
    else:
        model_recommendation = 'Invalid'
    
    
    #buy or not buy signal:
    if dateType == '1':
    
        #give the analyst a prediction
        signal = naive_bayes_model.predict(x_prediction_set)
        
        print()
        print('This is the buy signal:')
        
        #create a prediction variable
        predicition_value = signal[0] 
        
        if predicition_value == 0:
            buy_signal = 'Do NOT buy'

        
        #if the prediciton value is not equal to zero then it is equal to one, thus signaling a buying opportunity
        else:
            #assign the buy value to the variable
            buy_signal = 'Buy'

        
        #add the buy signal to the dictionary
        confusion_matrix_values = {'TP':[tp],'FP':[fp],'FN':[fn],'TN':[tn],'Precision WA':[precision],'Recall WA':[recall],'F1-Score WA':[f1_score],'Model Score %':[model_score],'Model Recommendation':[model_recommendation],'Buy Signal':[buy_signal]}
        
        return (confusion_matrix_values)

    #if datetype is not equal to 1 it is equal to 2 which means that the results will be the model accuracy and model recommendations
    else:
        #create the dictionary that will store the relevant values (buy signal is excluded because there isnt one to give)
        confusion_matrix_values = {'TP':[tp],'FP':[fp],'FN':[fn],'TN':[tn],'Precision WA':[precision],'Recall WA':[recall],'F1-Score WA':[f1_score],'Model Score %':[model_score],'Model Recommendation':[model_recommendation]}

#         print('This is the confusion matrix:')
#         print(confusion_matrix_values)

        #return confusion_matrix_values
        return (confusion_matrix_values)




### 7. Creating the User Interface Main Function

In [78]:
#revised main function
def main():
    
    #will help display the dataframes
    from IPython.display import display
    import pandas as pd
       
    #to ensure the model is only ran once at time, a 60 second timer will be placed in the main function
    import time
    
    #model trigger (model turns off if equal to false)
    modelValidation = True
    
    #add a print statement at the bottom to signal that the model has ended
    while  modelValidation == True:
                
        #there needs to be a tracker for how many calls have been used and when 
        total_api_calls = 0

        from datetime import datetime

        #ask for the API key
        api_q = input('What is your alphavantage API Key?:\n\n')
        api_q = 'TQ6TR98HVUPE3L9Z'
        print()

        #ask the data type
        data_type_q = input('What kind of data are you looking for?\n\nType the number in the input box:\n\n1. Upcoming Earnings Calendar (3 month horizon)\n2. Historical Earnings (for a specific company)\n3. Daily Adjusted Price History (for a specific company)\n4. Naive Bayes Prediction Model\n\n')

    
        if data_type_q == '1':

            print()
            date_of_interest = input('Pick a date that is within the next three months (yyyy-mm-dd):\n')
            print()

            #make the symbol equal to n/a because the function requires the parameter to be defined before performing what it needs to do
            symbol = 'N/A'

            #this function pulls the upcoming earnings calendar
            alphaEarningsCalendar  = alphaDataRetriever(api_q,data_type_q,symbol,date_of_interest)
            
            #api is called in the alphaEarningsCalendar 
            total_api_calls = total_api_calls + 1 

            print('Total API Calls: %i'%(total_api_calls))
            print()


            #if the dataframe does not contain any rows, that means its empty and no company is reporting earnings
            if alphaEarningsCalendar.shape[0] == 0:
                print('No publicly traded company is reporting their earnings on %s'%(date_of_interest))
                print('Please wait 60 seconds before running the model.')
#                 time.sleep(60)
                modelValidation = False

            else:

                #for some reason, if you put return dataframe, the function will not spit out the dataframe - use display instead
                #display
                display(alphaEarningsCalendar)

                print()
                
                
                #print disclaimers
                print('This is not investment advice.')
                print('Please wait 60 seconds before running the model again.')
                
                #sleep the model
#                 time.sleep(60)
                modelValidation = False
         
        #input 2 refers to the historical earnings table for a specific stock; it requires an api key and ticker symbol
        elif data_type_q == '2':
            print()
            symbol_q = input('Enter the ticker symbol:\n\n')
            print()
            
            #date_of_interest is not relevant when just pulling data
            date_of_interest = 'N/A'
      
            #this runs the historical earnings report for the symbol specified
            alphaHistoricalEarnings  = alphaDataRetriever(api_q,data_type_q,symbol_q,date_of_interest)
            
            #api is called in alphaHistoricalEarnings
            total_api_calls = total_api_calls + 1 
            
            #print disclaimers
            print('This is not investment advice.')
            print('Please wait 60 seconds before running the model again.')
            
            print('Total API Calls: %i'%(total_api_calls))

            #display first then sleep so that the analyst can look at something in the meantime
            display(alphaHistoricalEarnings)
            
            #sleep the model
#             time.sleep(60)
            
            modelValidation = False
    
        #input 3 refers to the daily adjusted price table for a specific stock; it requires an api key and ticker symbol
        elif data_type_q == '3':
            
            #ask the analyst what ticker they want
            print()
            symbol_q = input('Enter the ticker symbol:\n\n')
            print()
            
            #date_of_interest is not relevant when just pulling data because this function pulls most recent data
            date_of_interest = 'N/A'
            
            #runs the function that pulls daily adjusted prices for a specific stock
            alphaDailyAdjusted  = alphaDataRetriever(api_q,data_type_q,symbol_q,date_of_interest)
            
            #api is called
            total_api_calls = total_api_calls + 1 

            #disclaimer
            print('This is not investment advice.')
            print('Please wait 60 seconds before running the model again.')
            print()
            
            #print the total calls, this should be 1
            print('Total API Calls: %i'%(total_api_calls))
            print()
            
            #display first then sleep so that the analyst can look at something in the meantime
            display(alphaDailyAdjusted.head(15))
        
#             #sleep the model
#             time.sleep(60)
            
            modelValidation = False

################################################################################################################################################################################################################################################        
        #naive bayes prediction option
        elif data_type_q == '4':
            
            import math         
            print()
            
            #ask the analyst what data the model should do analysis on
            analysis_type = input('What type of analysis are you looking to do?:\n\n1.Individual Company\n\n2.Multi-Company\n\n')

            print()
            
            
            #the model must know whe 
            date_of_interest = input('Enter the date of interest:\n\n')
            print()
            
            #individual stock analysis
            if analysis_type == '1':

                #ask what stock they want analyzed
                ticker = input('Enter the ticker symbol:\n\n')
                print()

                #the historical earnings should be pulled first to see if the analysis can be done
                #switch the value of data_type_q to 2 so that the historical earnings table can be pulled via alphaDataRetriever
                data_type_q = '2'
                
                try:
                    
                    #run the alphaDataretriever to get the historical earnings for each symbol in the list
                    alphaHistoricalEarnings  = alphaDataRetriever(api_q,data_type_q,ticker,date_of_interest)
                    print()
#                     print('This dataframe contains %s\'s historical earnings report. The row pertains to either the most recent earnings date (if the date is in the past/the same as today) or the most upcoming earnings report date (if the date is in the future)'%(ticker))
#                     display(alphaHistoricalEarnings)
            
                except:
                    print('Function Broke. Check your inputs.')
                    #turn the model off
                    modelValidation = False
                #count the call
                total_api_calls = total_api_calls + 1


                #check to see if the company chosen has enough data to run a proper analysis on
                #the model should have at least 3 years worth of quarterly reports so that 2 years (8 quarters) can be used to train the model and 1 year (4 quarters) can be used to test
                if alphaHistoricalEarnings.shape[0] >= 12:
                    print(ticker,'has reported earnings',alphaHistoricalEarnings.shape[0],'times, as of the current date.')

                    #change the value of data_type_q to 3 to call the adjusted price function
                    data_type_q = '3'

                    #run the function that gets the daily adjusted prices for the ticker symbol in the loop
                    alphaDailyAdjusted  = alphaDataRetriever(api_q,data_type_q,ticker,date_of_interest)
                    
#                     print('This dataframe contains %s\'s 5,000 most recent daily adjusted prices:\n '%(ticker))
#                     display(alphaDailyAdjusted.head(15))

                    #count the call
                    total_api_calls = total_api_calls + 1

                    #change value of symbol so that the function can pull daily prices for SPY
                    symbol = 'SPY'

                    #the symbol here SPY will be constant and won't change
                    alphaSPY  = alphaDataRetriever(api_q,data_type_q,symbol,date_of_interest)
                    print()
#                     print('This dataframe contains SPY\'s most 5,000 most recent daily adjusted prices:\n')
#                     print(alphaSPY.head(15))

                    #this is the second call, but the SPY wont be called again for the remaining of the analysis
                    total_api_calls = total_api_calls + 1 
                
                
                    #the most recent date will be located in this cell
                    most_recent_date = alphaHistoricalEarnings.iloc[0,0]
                    
#                     #print the most recent earnings date (aka the first cell value of first row)
#                     print('%s\'s most recent/upcoming reporting date is:\n'%(ticker))
#                     print(most_recent_date)
#                     print()
                    #alphaHistoricalEarnings will return an earnings history report that includes the most upcoming earnings date
                    #if the date in the most upcoming earnings date happens to be the same as today and the time is after 4:01 (market close) then do the following:
                    
                    
                    import datetime 
                    
                    #current time instance
                    current_time = datetime.datetime.now() 
                    
                    #calculate this so that the model knows if the most recent/upcoming earnings is today
                    current_date = str(current_time.year) + '-' + str(current_time.month) + '-' + str(current_time.day)

                    #calculate this so that the model knows if the market has closed (after 4 PM)
                    current_hour = current_time.hour
                    
                    #if the most recent date in the historical earnings dataframe is equal to today's date and the market has closed then change the date_type and call the finhub function that has the most updated earnings information :
                    if most_recent_date == current_date and current_hour >= 17:
#                         print('test')
                        
                        date_type = '1'
                        
                        #ask the analyst for thier finhub api key
                        print()
                        
                        finhubapi = input('Enter your finhub API-Key:\n\n') 
                        print()
                                                
                        #ticker should be equal to empty space to get the most updated data point
                        ticker = ticker
                        #call the finhub function that will be used to create buying signals
                        finhub_function = finhubEarningsCalendar(finhubapi, date_of_interest,date_of_interest,ticker)
                        
                        #display
                        print()
                        print('This is %s\'s most updated earnings report:\n'%(ticker))
                        display(finhub_function)
                            
                        
                    
                    #if it is not the current date AND the market has not closed, then switch the date_type and assign the finhub function to 'N/A' because it's not needed
                    else:
                        date_type = '2'
                        finhub_function = 'N/A'    
    
    
                    #call the historicalEarningsTransformer to calculate the first 2 variables
                    historical_earnings_main = historicalEarningsTransformer(date_type,alphaHistoricalEarnings,finhub_function)
                    
                    #display
                    print()
#                     print('This is %s\'s most updated earnings table that contains the earnings_event and surprise_event variables that will be used in the analysis:\n'%(ticker))
#                     display(historical_earnings_main.head(15)) 

                    #this runs the first part of the model where the variables are transformed only if the api call count is within the 5 call limit
                    dailyAdustedPriceTransformer_main = dailyAdustedPriceTransformer(alphaDailyAdjusted,historical_earnings_main,alphaSPY)
                            
                    #display
#                     print()
#                     print('This is %s\'s most updated dataframe that contains the three price-oriented variables (intraday_event, adjusted_price_event, spy_event) that will be used in the analysis:\n'%(ticker))
#                     display(dailyAdustedPriceTransformer_main.head(15)) 
                    
#                     print()
                    #call the finalTransformation function that will get all the x and y variables ready
                    output_type = input('What type of output would you like?:\n\nLogic will display how the model variables were created, while Clean Variables will not:\n\n 1.Logic\n\n 2.Clean Variables\n\n')
                    print()
                    
                    #if the user picks logic
                    if output_type == '1':
                        #print the displays down to the order:
                        
                        #first run the historical earnings report for the stock 
                        print('1. Retrieve the historical earnings report for the stock:\n')
                        print('This dataframe contains %s\'s historical earnings report. The first row pertains to either the most recent earnings date (if the date is in the past/the same as today) or the upcoming earnings report date (if the date is in the future)'%(ticker))
                        display(alphaHistoricalEarnings.head(15))
                        print()
                        
                        #secondly
                        print('2. Insert the historical earnings report into the historicalEarningsTransformer function:\n')
                        print('This is %s\'s most updated earnings table that contains the earnings_event and surprise_event variables that will be used in the analysis:\n'%(ticker))
                        display(historical_earnings_main.head(15))
                        print()
                        
                        #thirdly
                        print('3. Retrieve the last 5,000 daily adjusted prices for %s in order to create the intraday_event and adjusted_price_event variables:\n'%(ticker))
                        display(alphaDailyAdjusted.head(15))
                        print()
                        
                        #fourth
                        print('4. Retrieve the last 5,000 daily adjusted prices for SPY in order to create the spy_event variable:\n')
                        display(alphaSPY.head(15))
                        print()
                        
                        #fifth
                        print('5. Connect both daily adjusted price dataframes (Stock Prices and SPY Prices) with the historical earnings report in order to calculate the x variables intraday_event, adjusted_price_event, spy_event:\n')
                        display(dailyAdustedPriceTransformer_main.head(15))
                        print()
                        
                        #sixth
                        print('6. Run the final tranformer function to generate the class variable:\n')
                        print()
  
                        
                        
                    
                    #parameters include : individual_hist_earn,both_dailyAdjustedPrices,daily_prices,output_type
                    finTransformationTransformer_main = finalTransformation(historical_earnings_main,dailyAdustedPriceTransformer_main,alphaDailyAdjusted,output_type,date_type)
                    print()
                    print('This is the dataframe that contains all the clean variables necessary for analysis:')
                    display(finTransformationTransformer_main.head(15))
                    
                    #run the model by calling the naivebayesPredictor function (returns a dictionary):
                    model_output = naiveBayesPredictor(finTransformationTransformer_main,date_type)

                    #convert the dictionary into a readable dataframe:
                    individual_analysis_results = pd.DataFrame.from_dict(model_output)
                    
                    #display the pretty dataframe
                    print()
                    print('Result dataframe:\n')
                    display(individual_analysis_results)
                    
                    #say bye
                    print('Thank you for using the model.')
                    print('Please wait 60 seconds before running the model again.')

                    #sleep the model
                    #time.sleep(60)
                
                    #turn model off
                    modelValidation = False
                
                #if the rows are less than 12 and not equal to 0 then:
                else:
                    
                    #print diagnosis
                    print('Company does not have enough reported earnings history to conduct a proper naive bayes prediction analysis.')
                    print('Please wait 60 seconds before running the model again.')

                    #sleep the model
#                     time.sleep(60)
                    
                    #turn the model off since the naive bayes cannot be done
                    modelValidation = False

#code works up to this point. Below is the multi-company analysis                    
############################################################################################################################
      
            #if the analysis type is not 1 then it is equal to 2 which refers to the multi-company analysis
            else:
                
                #big assumption: date_of_interest can only be the current date or plus three months because a past date will break the model
                #if the date of interest is equal to today and the time is after market close, then the model will assume that it's go time
                #if the date of interest is equal to today but the time is before market close, then the model will just give back model results
                #if the date of interest is not equal to today, then it has to be 
                
                #call the finalTransformation function that will get all the x and y variables ready
                output_type = input('What type of output would you like?:\n\nLogic will display how the model variables were created, while Clean Variables will not:\n\n 1.Logic\n\n 2.Clean Variables\n\n')
                print()
                
                
                
                import datetime                 
                
                # using now() to get current time 
                current_time = datetime.datetime.now() 
#                 print(current_time)

                #calculate this so that the model knows if the most recent/upcoming earnings is today
                current_date = str(current_time.year) + '-' + str(current_time.month) + '-' + str(current_time.day)
#                 print(current_date)

                #calculate this so that the model knows if the market has closed (after 4 PM)
                current_hour = current_time.hour
#                 print(current_hour)
################################################################################################################################################################################################################################################################                
                #if the date of interest is the same as the current date and the market is closed, then run the finhub function
                #finhub is the most updated API and it will have the actual companies that reported earnings
                #the earnings calendar gives estimates of when the stock is going to report earnings and when they are wrong, finhub is used fill in the gap as a discrepancy check
                if date_of_interest == current_date and current_hour >= 17:
                                       
#                     print('test')
                    #change the data_type because the model is in its prime time mode, ready to make a prediciton for the next trading date
                    date_type = '1'

                    #ask the analyst for thier finhub api key
                    print()
                    finhubapi = input('Enter your finhub API-Key:\n\n') 

                    #empty space as the value for ticker will return all the stocks that reported earnings on that date
                    ticker = ''

                    #call the finhub function that will be used to create buying signals
                    finhub_function = finhubEarningsCalendar(finhubapi, date_of_interest,date_of_interest,ticker)

                    #debug
                    print('Below is the finhub function')
                    display(finhub_function)

                    #loop through the symbol column so that the model knows what stocks to analyze and give a buy signal to
                    #create the list of symbols that will be used in the for loop
                    list_of_ticker_symbols =  finhub_function.symbol.tolist()
                        

#                         #get the length of the list to determine how many stocks are in your dataset
#                         stock_count = len(list_of_ticker_symbols)    
                        
#                         #print stock count
#                         print('Stock Count:\n',stock_count)

#                         #for a naive bayes analyis, each stock will need its earnings history and daily adjusted prices 
#                         stock_api_calls = stock_count * 3

#                         #5 api calls per minute
#                         api_calls_per_minute = 5

#                         #how many sets of 5 api calls/min need to be done in order to do an analysis on all of the stocks
#                         #this will round up to the nearest integer
#                         #this essentially how long (in minutes) the model will take to run an analysis on all of the stocks on the earnings calendar for the specified date
#                         api_sets  = math.ceil(stock_api_calls / api_calls_per_minute)
                        
################################################################################################################################################################################################################################################################
                #if it is not the current date AND the market has not closed, then switch the date_type and assign the finhub function to 'N/A' because it's not needed                
                #model results, NO buy signal
                #different thought process here
                else:
                    
                    #not go time
                    date_type = '2'
                    finhub_function = 'N/A'    
                    
                    #code for the earnings calendar
                    data_type_q = '1'
                    
                    #change symbol value
                    symbol = ''
                    
                    #run the alphaDataRetriever to get the earnings calendar 
                    alphaEarningsCalendar  = alphaDataRetriever(api_q,data_type_q,symbol,date_of_interest)
#                     print('This is the alpha earnings calendar:\n')
#                     print('Symbol: ',symbol)
#                     print('Date of Interest: ',date_of_interest)
#                     print('Data Type: ',data_type_q)
#                     print()
#                     display(alphaEarningsCalendar)
                    
                    print()
                    print('Model is sleeping so that it can reset')
                    #sleep model so that the count resets
                    time.sleep(60)
                    
                    #reference the list from the alpha earnings calendar
                    list_of_ticker_symbols = alphaEarningsCalendar.symbol.tolist()
#                     print('List of Stocks from the alphaEarningsCalendar: ',list_of_ticker_symbols)
################################################################################################################################################################################################################################################################                      
                #Calculate the api information:        
                        
                #get the length of the list to determine how many stocks are in your dataset
                stock_count = len(list_of_ticker_symbols)    

                #print stock count
                print()
                print('Stock Count:',stock_count)
                print()

                #for a naive bayes analyis, each stock will need its earnings history and daily adjusted prices and the SPY call
                stock_api_calls = stock_count * 3

                #5 api calls per minute
                api_calls_per_minute = 5

                #how many sets of 5 api calls/min need to be done in order to do an analysis on all of the stocks
                #this will round up to the nearest integer
                #this essentially how long (in minutes) the model will take to run an analysis on all of the stocks on the earnings calendar for the specified date
                api_sets  = math.ceil(stock_api_calls / api_calls_per_minute)        
                        

                #12 min should be the longest an analyst waits for an analysis, this equal to 30 stocks that have 3 dataframes each 
                if stock_count > 30:

                    #only get the first 30 symbols
                    list_of_ticker_symbols = list_of_ticker_symbols[0:15]
                    
                    #15 stocks * 3 dataframes = 45
                    stock_api_calls = 15 * 3
                    
                    #api_sets are set to 9 mintutes
                    api_sets  = math.ceil(stock_api_calls / api_calls_per_minute)  
                
                    #user experience
                    print('There are %i companies reporting earings on the specified date. Analysis will only be provided for the first 15 companies in the list. This should take about %i minutes. Please wait patiently.'%(stock_count,api_sets))

                
                
                #if there is 30 or less stocks in the list, keep the symbol list the same
                else:
                    #keep the same
                    list_of_ticker_symbols = list_of_ticker_symbols
                    stock_api_calls = stock_api_calls
                                    
                    #tell the analyst how long the analysis is going to take
                    print('The model will take about %i minutes to run. Please wait patiently.'%(api_sets))



                print()
                #list of symbols stay the same
                list_of_ticker_symbols  = list_of_ticker_symbols

                #stock_api_calls stay the same
                stock_api_calls = stock_api_calls 
                print()
                print('There are a total of %i stock calls'%(stock_api_calls))
################################################################################################################################################################################################################
                #for loop begins here:
                #this list will contain the confusion matrix for each stock in the for loop
                #list of dictionaries
                model_result_list = []

                #for each ticker in the list of symbols, run each ticker through the functions
                for ticker in list_of_ticker_symbols:

#                     print()
#                     print('This is the stock_api_call at each loop:')
#                     print()
#                     print(stock_api_calls)
                    print()
                    print('This is the ticker:')
                    print()
                    print(ticker)
                    print()


                    #change value of data_type_q so that the function can pull daily prices for SPY
                    data_type_q = '3'
                    print(date_of_interest)
                    #the symbol here SPY will be constant and won't change
                    alphaSPY  = alphaDataRetriever(api_q,data_type_q,'SPY',date_of_interest)
                    print()
#                     print('This is the alphaSPY')
#                     display(alphaSPY.head(15))

                    #this is the second call
                    total_api_calls = total_api_calls + 1 

#                     #subtract the call from the stock count
#                     stock_api_calls = stock_api_calls - 1


                    if total_api_calls == api_calls_per_minute:
                        total_api_calls = 0
                        print('Model has been put to sleep after the alpha spy')
                        time.sleep(60)

                    else:
                        total_api_calls = total_api_calls


                    #assign symbol the same value as ticker so that the tickers in the column are inputted into the historical earnings function
                    symbol = ticker    

                    #switch the value of data_type_q to 2 so that the historical earnings table can be pulled via alphaDataRetriever
                    data_type_q = '2'

                    #run the alphaDataretriever to get the historical earnings for each symbol in the list

                    try:
#                         print(symbol)
                        print()
                        alphaHistoricalEarnings  = alphaDataRetriever(api_q,data_type_q,symbol,date_of_interest)
                        print()
#                         print('This is the historical earnings dataframe')
#                         display(alphaHistoricalEarnings)
                    except:
                        print()
                        print(ticker)
                        print('An error has occured in the historical earnings function. Check inputs.')
                        #skip to the next ticker and run analysis on it
                        print('This will fuck up the rest of the functions')
#                         total_api_calls = total_api_calls -1
                        
                        continue
        
                    

                    #the historical earnings are called 
                    total_api_calls = total_api_calls + 1

#                     #subtract the call from the stock count
#                     stock_api_calls = stock_api_calls - 1


                    #if the total api calls are 5 then a full set has been completed and it can be subtracted from the total sets needed to gather the information
                    if total_api_calls == api_calls_per_minute:

                        #reset the number of total api calls
                        total_api_calls = 0
                        print()
                        print('Model has been put to sleep after historical earnings call')
                        #sleep the model because an additional api call will break the model
                        time.sleep(60)

                    #if the total api calls isn't equal to 5 then keep the value the same
                    else:

                        total_api_calls = total_api_calls 
################################################################################################################################################################################################################
                    #this section analyzes the stock only if there is enough training data available
    
                    #still within the for loop
                    #the model should have at least 3 years worth of quarterly reports so that 2 years (8 quarters) can be used to train the model and 1 year (4 quarters) can be used to test
                    #if the model has 12 or more instances, then conduct the analysis and provide a buying signal
                    if alphaHistoricalEarnings.shape[0] >= 12:
                        print(symbol,'has reported earnings',alphaHistoricalEarnings.shape[0],'times, as of the current date.')
                        print()

                        #change the value of data_type_q to 3 to call the adjusted price function
                        data_type_q = '3'

                        #run the function that gets the daily adjusted prices for the ticker symbol in the loop
                        try:
#                             print('This is the daily adjusted dataframe:')
#                             print('Symbol: ',symbol)
#                             print('Date of Interest: ',date_of_interest)
#                             print('Daily Adjusted Dataframe:\n')
                            
                            alphaDailyAdjusted  = alphaDataRetriever(api_q,data_type_q,symbol,date_of_interest)
#                             print('This is the daily adjusted price for 
                            
#                             print()
                            
                        except:
#                             total_api_calls = total_api_calls -1
                            print('symbol:',symbol)
                            print('date_of_interest',date_of_interest)
                            print()

                    
                    
                    
                        
#                         print('An error has occured in the alphaDailyAdjusted function. Check inputs.')
                        #skip to the next ticker and run analysis on it
#                         print('skip to the next ticker and run analysis on it')
#                         continue


                        #count the call
                        total_api_calls = total_api_calls + 1

                        #subtract the set from the stock count
#                         stock_api_calls = stock_api_calls - 1


                        #if the total api calls are 5 then a full set has been completed and it can be subtracted from the total sets needed to gather the information
                        if total_api_calls == api_calls_per_minute:
                            #reset the number of total api calls
                            total_api_calls = 0
                            print()
                            print('model has been put to sleep after the daily adjusted call')
                            #sleep the model because an additional api call will break the model
                            time.sleep(60)

                        #keep as is if the total api calls isnt equal to 5 
                        else:

                            total_api_calls = total_api_calls


                        #make sure to call the finhub_function for each stock

              

                        try:
#                             #call the finhub function that will be used to create buying signals
#                             finhub_function = finhubEarningsCalendar(finhubapi, date_of_interest,date_of_interest,ticker)
                            
                            if date_type = '1':
#                                 call the finhub function that will be used to create buying signals
                               finhub_function = finhubEarningsCalendar(finhubapi, date_of_interest,date_of_interest,ticker)
            
                                #call the historicalEarningsTransformer to calculate the first 2 variables
                                historical_earnings_main = historicalEarningsTransformer(date_type,alphaHistoricalEarnings,finhub_function)
                            
                            else:
                                #call the historicalEarningsTransformer to calculate the first 2 variables
                                historical_earnings_main = historicalEarningsTransformer(date_type,alphaHistoricalEarnings,finhub_function)
                      
                            
                            #display
                            print()
#                             print('Below is the historical main function that returns the two x variables')
#                             display(historical_earnings_main.head(15)) 
                        except:
                            print()
                            print('An error has occured in the historicalEarningsTransformer function. Check inputs.')
                            print('')
                            print('finhub api',finhubapi)
                            print()
                            print('Display historical earnings')
                            print()
                            display(alphaHistoricalEarnings.head(5))
                            print()
                            print('finhub function',finhub_function)
                            
                            #skip to the next ticker and run analysis on it
                            print('skip to the next ticker and run analysis on it')
                            
                            continue



                        try:
                            #this runs the first part of the model where the variables are transformed only if the api call count is within the 5 call limit
                            dailyAdustedPriceTransformer_main = dailyAdustedPriceTransformer(alphaDailyAdjusted,historical_earnings_main,alphaSPY)

                            #display
                            print()
#                             print('This is the dataframe with the remaining 3 variables that pertain to the earnings data')
#                             display(dailyAdustedPriceTransformer_main.head(15)) 
                        except:
                            print()
                            #skip to the next ticker and run analysis on it
                            print('skip to the next ticker and run analysis on it')
                            continue

                            print('An error has occured in the dailyAdustedPriceTransformer function. Check inputs.')


                        #call the finalTransformation function that will get all the x and y variables ready
#                             output_type = input('What type of output would you like?:\n\nLogic will display how the model variables were created, while Clean Variables will not:\n\n 1.Logic\n\n 2.Clean Variables\n\n')

#                                         output_type = '2'

                        #if the user chooses output type logic, then print the following statements before running final transformer
                        if output_type == '1':

                            #print the following statements:
                            #first run the historical earnings report for the stock 
                            print('1. Retrieve the historical earnings report for the stock:\n')
                            print('This dataframe contains %s\'s historical earnings report. The first row pertains to either the most recent earnings date (if the date is in the past/the same as today) or the upcoming earnings report date (if the date is in the future)'%(ticker))
                            print()
                            try:
                                display(alphaHistoricalEarnings.head(15))
                                print()
                            except:
                                print('Does not exist.')
                            #secondly
                            print('2. Insert the historical earnings report into the historicalEarningsTransformer function:\n')
                            print('This is %s\'s most updated earnings table that contains the earnings_event and surprise_event variables that will be used in the analysis:\n'%(ticker))

                            try:
                                display(historical_earnings_main.head(15))
                                print()
                            except:
                                print('Does not exist.')

                            #thirdly
                            print('3. Retrieve the last 5,000 daily adjusted prices for %s in order to create the intraday_event and adjusted_price_event variables:\n'%(ticker))

                            try:
                                display(alphaDailyAdjusted.head(15))
                                print()
                            except:
                                print('Does not exist.')

                            #fourth

                            try:
                                print('4. Retrieve the last 5,000 daily adjusted prices for SPY in order to create the spy_event variable:\n')
                                display(alphaSPY.head(15))
                                print()
                            except:
                                print('Does not exist.')

                            #fifth
                            try:
                                print('5. Connect both daily adjusted price dataframes (Stock Prices and SPY Prices) with the historical earnings report in order to calculate the x variables intraday_event, adjusted_price_event, spy_event:\n')
                                display(dailyAdustedPriceTransformer_main.head(15))
                                print()
                                
                            except:
                                print('Does not exist.')

                            #sixth
                            print('6. Run the final tranformer function to generate the class variable:\n')
                            print()
                            
#                             try:
                                
#                           #see what's breaking specifically     

                            print('Run the analysis with just clean variables')
                            #parameters include : individual_hist_earn,both_dailyAdjustedPrices,daily_prices,output_type
                            finTransformationTransformer_main = finalTransformation(historical_earnings_main,dailyAdustedPriceTransformer_main,alphaDailyAdjusted,output_type,date_type)
                            display(finTransformationTransformer_main)
#                             else:
#                                 print('No variables to display.')
                                #debug
#                                 print()
#                                 print()
                            
                            
                            
                            try:
                                #run the model by calling the naivebayesPredictor function (returns a dictionary):
                                model_output = naiveBayesPredictor(finTransformationTransformer_main,date_type)

                                #add the symbol to the dictionary values 
                                model_output['symbol'] = symbol 

                                model_result_list.append(model_output)

                                #convert the dictionary into a readable dataframe:
                                individual_analysis_results = pd.DataFrame.from_dict(model_output)

                                #display the pretty dataframe
                                display(individual_analysis_results)
                            except:
                                print('No predicitive output to display.')
                                print()




                        
                        #if the output type is equal to 2 aka clean
                        else:
                            
                            try:
                                
                                print('Run the analysis with just clean variables')
                                #parameters include : individual_hist_earn,both_dailyAdjustedPrices,daily_prices,output_type
                                finTransformationTransformer_main = finalTransformation(historical_earnings_main,dailyAdustedPriceTransformer_main,alphaDailyAdjusted,output_type,date_type)

                            except:
                                print('No variables to display.')
                                print()

                        
#                         #if the output is equal to 2 aka clean variables:
#                         else:
#                 print(model_result_list)            
                        
                        
                        
#                         #parameters include : individual_hist_earn,both_dailyAdjustedPrices,daily_prices,output_type
#                         finTransformationTransformer_main = finalTransformation(historical_earnings_main,dailyAdustedPriceTransformer_main,alphaDailyAdjusted,output_type,date_type)
#                         print()
# #                         print('This is the dataframe that contains all the clean variables necessary for analysis:')
# #                         display(finTransformationTransformer_main.head(15))

#                         try:
#                             print('This is the dataframe that contains all the clean variables necessary for analysis:')
#                             display(finTransformationTransformer_main.head(15))

#                         except:
# #                             print('No variables to display.')
# #                             print()

#                         try:
#                             #run the model by calling the naivebayesPredictor function (returns a dictionary):
#                             model_output = naiveBayesPredictor(finTransformationTransformer_main,date_type)
                            
#                             #add the symbol to the dictionary values 
#                             model_output['symbol'] = symbol 

#                             model_result_list.append(model_output)

#                             #convert the dictionary into a readable dataframe:
#                             individual_analysis_results = pd.DataFrame.from_dict(model_output)

#                             #display the pretty dataframe
#                             display(individual_analysis_results)
#                         except:
#                             print('No predicitive output to display.')
#                             print()

#                         #add the symbol to the dictionary values 
#                         model_output['symbol'] = symbol 

#                         model_result_list.append(model_output)

#                         #convert the dictionary into a readable dataframe:
#                         individual_analysis_results = pd.DataFrame.from_dict(model_output)

#                         #display the pretty dataframe
#                         display(individual_analysis_results)

################################################################################################################################################################################################################
                    #if the stock does not have enough data then skip and go on to the next one
                    else:
                        print('This stock does not have enough data to conduct a proper naive bayes analysis.')

                        #if there are not enough data points for a proper analysis, that means that this stock will not pull the remaining 2 dataframes
                        #subtract the api call count by 2
#                         stock_api_calls = stock_api_calls - 2
#                         print(stock_api_calls)

                                    
                                
                                


                #at this point, the for loop has ended and the results of the multi-company analysis shoudl be displayed
                #say bye
#                 print()
#                 print(model_result_list)
                print()
                print('Model performance across all the stock in the list:\n')
                multi_analysis_results = pd.DataFrame.from_dict(model_result_list)
                display(multi_analysis_results)
                print('Thank you for using the model.')
                print('Please wait 60 seconds before running the model again.')

                #sleep the model
                time.sleep(60)

                #turn model off
                modelValidation = False     

                        
#############################################################################################################################################################################################################                        
        #this else was put here to incase the analyst put invalid option to the first question of data of interest
        else:
            
            print('Invalid input. Please run the model again.')
            #turn the model off since the user put an invalid option
            modelValidation = False
        
        print()

SyntaxError: invalid syntax (<ipython-input-78-e97408877fb9>, line 694)

## Showcasing Model Outputs

### 1. Upcoming Earnings Calendar

In [34]:
if __name__=="__main__":
    main()        

#alphavantage API key
#TQ6TR98HVUPE3L9Z

#finhub API key
#c5d5iiqad3i9ue38pn9g

#the present date and time is 10/21/2021 at 5 PM

What is your alphavantage API Key?:

TQ6TR98HVUPE3L9Z

What kind of data are you looking for?

Type the number in the input box:

1. Upcoming Earnings Calendar (3 month horizon)
2. Historical Earnings (for a specific company)
3. Daily Adjusted Price History (for a specific company)
4. Naive Bayes Prediction Model

1

Pick a date that is within the next three months (yyyy-mm-dd):
2021-10-22

Total API Calls: 1



,symbol,name,reportDate,estimate
150,AIMC,Altra Industrial Motion Corp,2021-10-22,0.82
218,ALV,Autoliv Inc,2021-10-22,0.87
483,AXP,American Express Company,2021-10-22,1.8
963,CLF,Cleveland-Cliffs Inc,2021-10-22,2.26
1366,DSKE,Daseke Inc,2021-10-22,0.32
1667,FHB,First Hawaiian INC,2021-10-22,0.48
1936,GNTX,Gentex Corp,2021-10-22,0.39
2045,HCA,HCA Healthcare Inc,2021-10-22,4
2116,HON,Honeywell International Inc,2021-10-22,1.99
2883,MNR,Monmouth Real Estate Investment Corp - Class A,2021-10-22,0.07



This is not investment advice.
Please wait 60 seconds before running the model again.



### 2. Historical Earnings Report (Specific Stock)


In [35]:
if __name__=="__main__":
    main()        

#alphavantage API key
#TQ6TR98HVUPE3L9Z

#finhub API key
#c5d5iiqad3i9ue38pn9g

#the present date and time is 10/21/2021 at 5 PM

What is your alphavantage API Key?:

TQ6TR98HVUPE3L9Z

What kind of data are you looking for?

Type the number in the input box:

1. Upcoming Earnings Calendar (3 month horizon)
2. Historical Earnings (for a specific company)
3. Daily Adjusted Price History (for a specific company)
4. Naive Bayes Prediction Model

2

Enter the ticker symbol:

AIMC

This is not investment advice.
Please wait 60 seconds before running the model again.
Total API Calls: 1


,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage
0,2021-07-23,0.89,0.815,0.075,9.2025
1,2021-04-29,0.86,0.7633,0.0967,12.6687
2,2021-02-12,0.76,0.69,0.07,10.1449
3,2020-10-23,0.87,0.46,0.41,89.1304
4,2020-07-24,0.6,0.31,0.29,93.5484
5,2020-04-30,0.65,0.54,0.11,20.3704
6,2020-02-13,0.66,0.6433,0.0167,2.596
7,2019-10-24,0.69,0.6633,0.0267,4.0253
8,2019-07-25,0.71,0.7867,-0.0767,-9.7496
9,2019-04-26,0.8,0.7225,0.0775,10.7266


### 3. Daily Adjusted Price Report (Specific Stock)

In [38]:
if __name__=="__main__":
    main()        

#alphavantage API key
#TQ6TR98HVUPE3L9Z

#finhub API key
#c5d5iiqad3i9ue38pn9g

#the present date and time is 10/21/2021 at 5 PM

What is your alphavantage API Key?:

TQ6TR98HVUPE3L9Z

What kind of data are you looking for?

Type the number in the input box:

1. Upcoming Earnings Calendar (3 month horizon)
2. Historical Earnings (for a specific company)
3. Daily Adjusted Price History (for a specific company)
4. Naive Bayes Prediction Model

3

Enter the ticker symbol:

AIMC

This is not investment advice.
Please wait 60 seconds before running the model again.

Total API Calls: 1



,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,Intraday_Returns,Weekly_Median_Return,trading_date
0,56.60,57.150,55.890,57.00,57.00,253571,0.0,1.0,0.7067,0.5395,2021-10-21
1,55.97,57.380,55.970,57.01,57.01,150512,0.0,1.0,1.8581,0.1787,2021-10-20
2,55.95,56.380,55.250,56.05,56.05,145147,0.0,1.0,0.1787,-0.4873,2021-10-19
3,54.24,55.750,54.240,55.59,55.59,138822,0.0,1.0,2.4889,-0.5667,2021-10-18
4,55.41,55.679,54.090,55.14,55.14,354577,0.0,1.0,-0.4873,-0.5667,2021-10-15
5,53.75,54.310,53.350,54.04,54.04,231975,0.0,1.0,0.5395,-0.5667,2021-10-14
6,54.37,54.370,52.860,52.99,52.99,193745,0.0,1.0,-2.5382,-0.5667,2021-10-13
7,54.70,55.085,53.970,54.39,54.39,118328,0.0,1.0,-0.5667,-0.3206,2021-10-12
8,56.16,56.250,54.400,54.56,54.56,170354,0.0,1.0,-2.8490,-0.3206,2021-10-11
9,54.88,56.260,53.590,55.82,55.82,209190,0.0,1.0,1.7128,-0.3206,2021-10-08


### 4.1. Naive Bayes Prediction Model (Specific Stock | Future Date | Logical Display)

In [42]:
if __name__=="__main__":
    main()        

#alphavantage API key
#TQ6TR98HVUPE3L9Z

#finhub API key
#c5d5iiqad3i9ue38pn9g
#the present date and time is 10/21/2021 at 5 PM

What is your alphavantage API Key?:

TQ6TR98HVUPE3L9Z

What kind of data are you looking for?

Type the number in the input box:

1. Upcoming Earnings Calendar (3 month horizon)
2. Historical Earnings (for a specific company)
3. Daily Adjusted Price History (for a specific company)
4. Naive Bayes Prediction Model

4

What type of analysis are you looking to do?:

1.Individual Company

2.Multi-Company

1

Enter the date of interest:

2021-10-22

Enter the ticker symbol:

AIMC


AIMC has reported earnings 59 times, as of the current date.


What type of output would you like?:

Logic will display how the model variables were created, while Clean Variables will not:

 1.Logic

 2.Clean Variables

1

1. Retrieve the historical earnings report for the stock:

This dataframe contains AIMC's historical earnings report. The first row pertains to either the most recent earnings date (if the date is in the past/the same as today) or the upcoming earnings report date (if the date is in the future

,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage
0,2021-07-23,0.89,0.815,0.075,9.2025
1,2021-04-29,0.86,0.7633,0.0967,12.6687
2,2021-02-12,0.76,0.69,0.07,10.1449
3,2020-10-23,0.87,0.46,0.41,89.1304
4,2020-07-24,0.6,0.31,0.29,93.5484
5,2020-04-30,0.65,0.54,0.11,20.3704
6,2020-02-13,0.66,0.6433,0.0167,2.596
7,2019-10-24,0.69,0.6633,0.0267,4.0253
8,2019-07-25,0.71,0.7867,-0.0767,-9.7496
9,2019-04-26,0.8,0.7225,0.0775,10.7266



2. Insert the historical earnings report into the historicalEarningsTransformer function:

This is AIMC's most updated earnings table that contains the earnings_event and surprise_event variables that will be used in the analysis:



,column_of_interest,reportedEPS,estimatedEPS,surprise,surprisePercentage,earnings_event,surprise_event
0,2021-07-23,0.89,0.8150,0.075,9.2025,1,1
1,2021-04-29,0.86,0.7633,0.0967,12.6687,1,1
2,2021-02-12,0.76,0.6900,0.07,10.1449,0,1
3,2020-10-23,0.87,0.4600,0.41,89.1304,1,1
4,2020-07-24,0.60,0.3100,0.29,93.5484,0,1
5,2020-04-30,0.65,0.5400,0.11,20.3704,0,1
6,2020-02-13,0.66,0.6433,0.0167,2.5960,0,1
7,2019-10-24,0.69,0.6633,0.0267,4.0253,0,1
8,2019-07-25,0.71,0.7867,-0.0767,-9.7496,0,0
9,2019-04-26,0.80,0.7225,0.0775,10.7266,1,1



3. Retrieve the last 5,000 daily adjusted prices for AIMC in order to create the intraday_event and adjusted_price_event variables:



...

,earnings_event,surprise_event,adjusted_price_event,intraday_event,spy_event,class_variable
0,1,1,0,0,1,0
1,1,1,1,0,1,0
2,0,1,1,0,1,1
3,1,1,1,1,1,0
4,0,1,1,0,1,0
5,0,1,0,0,0,0
6,0,1,1,1,1,0
7,0,1,1,0,0,1
8,0,0,0,0,0,0
9,1,1,1,1,1,0



Result dataframe:



C:\Users\Papi\Documents\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,TP,FP,FN,TN,Precision WA,Recall WA,F1-Score WA,Model Score %,Model Recommendation
0,0,1,0,3,0.5625,0.75,0.642857,75.0,Invalid


Thank you for using the model.
Please wait 60 seconds before running the model again.



### 4.2. Naive Bayes Prediction Model (Specific Stock | Future Date | Clean Display)

In [44]:
if __name__=="__main__":
    main()        

#alphavantage API key
#TQ6TR98HVUPE3L9Z

#finhub API key
#c5d5iiqad3i9ue38pn9g
#the present date and time is 10/21/2021 at 5 PM

What is your alphavantage API Key?:

TQ6TR98HVUPE3L9Z

What kind of data are you looking for?

Type the number in the input box:

1. Upcoming Earnings Calendar (3 month horizon)
2. Historical Earnings (for a specific company)
3. Daily Adjusted Price History (for a specific company)
4. Naive Bayes Prediction Model

4

What type of analysis are you looking to do?:

1.Individual Company

2.Multi-Company

1

Enter the date of interest:

2021-10-21

Enter the ticker symbol:

AIMC


AIMC has reported earnings 59 times, as of the current date.


What type of output would you like?:

Logic will display how the model variables were created, while Clean Variables will not:

 1.Logic

 2.Clean Variables

2

Clean Variables:


This is the dataframe that contains all the clean variables necessary for analysis:


,earnings_event,surprise_event,adjusted_price_event,intraday_event,spy_event,class_variable
0,1,1,0,0,1,0
1,1,1,1,0,1,0
2,0,1,1,0,1,1
3,1,1,1,1,1,0
4,0,1,1,0,1,0
5,0,1,0,0,0,0
6,0,1,1,1,1,0
7,0,1,1,0,0,1
8,0,0,0,0,0,0
9,1,1,1,1,1,0



Result dataframe:



C:\Users\Papi\Documents\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,TP,FP,FN,TN,Precision WA,Recall WA,F1-Score WA,Model Score %,Model Recommendation
0,0,1,0,3,0.5625,0.75,0.642857,75.0,Invalid


Thank you for using the model.
Please wait 60 seconds before running the model again.



### 4.3 Naive Bayes Prediction Model (Specific Stock | Current Date | Logical Display)


In [57]:
if __name__=="__main__":
    main()        

#alphavantage API key
#TQ6TR98HVUPE3L9Z

#finhub API key
#c5d5iiqad3i9ue38pn9g

#the present date and time is 10/21/2021 at 5 PM
#HRI reported earnings today and now that its after market close, the model will make a prediction for tomorrow

What is your alphavantage API Key?:

TQ6TR98HVUPE3L9Z

What kind of data are you looking for?

Type the number in the input box:

1. Upcoming Earnings Calendar (3 month horizon)
2. Historical Earnings (for a specific company)
3. Daily Adjusted Price History (for a specific company)
4. Naive Bayes Prediction Model

4

What type of analysis are you looking to do?:

1.Individual Company

2.Multi-Company

1

Enter the date of interest:

2021-10-21

Enter the ticker symbol:

HRI


HRI has reported earnings 22 times, as of the current date.


Enter your finhub API-Key:

c5d5iiqad3i9ue38pn9g


This is HRI's most updated earnings report:



,symbol,reportedDate,hour,reportedEPS,estimatedEPS,surprise,surprisePercentage
0,HRI,2021-10-21,bmo,2.38,2.3836,-0.0036,-0.151



What type of output would you like?:

Logic will display how the model variables were created, while Clean Variables will not:

 1.Logic

 2.Clean Variables

1

1. Retrieve the historical earnings report for the stock:

This dataframe contains HRI's historical earnings report. The first row pertains to either the most recent earnings date (if the date is in the past/the same as today) or the upcoming earnings report date (if the date is in the future)


,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage
0,2021-10-21,None,2.37,None,None
1,2021-07-22,1.57,1.2433,0.3267,26.2768
2,2021-04-22,1.1,0.2543,0.8457,332.56
3,2021-02-18,1.35,1.0288,0.3212,31.2208
4,2020-10-22,1.35,0.8025,0.5475,68.2243
5,2020-07-23,0.25,-0.595,0.845,142.0168
6,2020-04-23,0.04,-0.17,0.21,123.5294
7,2020-02-27,1.33,1.56,-0.23,-14.7436
8,2019-10-23,1.48,1.7771,-0.2971,-16.7182
9,2019-08-01,0.5222,0.3314,0.1908,57.5739



2. Insert the historical earnings report into the historicalEarningsTransformer function:

This is HRI's most updated earnings table that contains the earnings_event and surprise_event variables that will be used in the analysis:



...

,earnings_event,surprise_event,adjusted_price_event,intraday_event,spy_event,class_variable
0,1,0,1,1,1,0
1,1,1,1,0,1,0
2,0,1,1,1,0,1
3,0,1,1,0,1,0
4,1,1,1,0,1,1
5,1,1,1,1,0,0
6,0,1,0,1,0,0
7,0,0,0,1,0,0
8,1,0,0,1,1,0
9,1,1,0,1,0,0



This is the buy signal:

Result dataframe:



C:\Users\Papi\Documents\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,TP,FP,FN,TN,Precision WA,Recall WA,F1-Score WA,Model Score %,Model Recommendation,Buy Signal
0,0,2,0,2,0.25,0.5,0.333333,50.0,Invalid,Do NOT buy


Thank you for using the model.
Please wait 60 seconds before running the model again.



### 4.4 Naive Bayes Prediction Model (Specific Stock | Current Date | Clean Display)

In [58]:
if __name__=="__main__":
    main()        

#alphavantage API key
#TQ6TR98HVUPE3L9Z

#finhub API key
#c5d5iiqad3i9ue38pn9g

#the present date and time is 10/21/2021 at 5 PM
#HRI reported earnings today and now that its after market close, the model will make a prediction for tomorrow

What is your alphavantage API Key?:

TQ6TR98HVUPE3L9Z

What kind of data are you looking for?

Type the number in the input box:

1. Upcoming Earnings Calendar (3 month horizon)
2. Historical Earnings (for a specific company)
3. Daily Adjusted Price History (for a specific company)
4. Naive Bayes Prediction Model

4

What type of analysis are you looking to do?:

1.Individual Company

2.Multi-Company

1

Enter the date of interest:

2021-10-21

Enter the ticker symbol:

HRI


HRI has reported earnings 22 times, as of the current date.


Enter your finhub API-Key:

c5d5iiqad3i9ue38pn9g


This is HRI's most updated earnings report:



,symbol,reportedDate,hour,reportedEPS,estimatedEPS,surprise,surprisePercentage
0,HRI,2021-10-21,bmo,2.38,2.3836,-0.0036,-0.151



What type of output would you like?:

Logic will display how the model variables were created, while Clean Variables will not:

 1.Logic

 2.Clean Variables

2

Clean Variables:


This is the dataframe that contains all the clean variables necessary for analysis:


,earnings_event,surprise_event,adjusted_price_event,intraday_event,spy_event,class_variable
0,1,0,1,1,1,0
1,1,1,1,0,1,0
2,0,1,1,1,0,1
3,0,1,1,0,1,0
4,1,1,1,0,1,1
5,1,1,1,1,0,0
6,0,1,0,1,0,0
7,0,0,0,1,0,0
8,1,0,0,1,1,0
9,1,1,0,1,0,0



This is the buy signal:

Result dataframe:



C:\Users\Papi\Documents\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,TP,FP,FN,TN,Precision WA,Recall WA,F1-Score WA,Model Score %,Model Recommendation,Buy Signal
0,0,2,0,2,0.25,0.5,0.333333,50.0,Invalid,Do NOT buy


Thank you for using the model.
Please wait 60 seconds before running the model again.



### 5.1. Naive Bayes Prediction Model (Multi-Stock | Future Date | Logical Display)

In [31]:
if __name__=="__main__":
    main()        

#alphavantage API key
#TQ6TR98HVUPE3L9Z

#finhub API key
#c5d5iiqad3i9ue38pn9g
#not complete

What is your alphavantage API Key?:

TQ6TR98HVUPE3L9Z

What kind of data are you looking for?

Type the number in the input box:

1. Upcoming Earnings Calendar (3 month horizon)
2. Historical Earnings (for a specific company)
3. Daily Adjusted Price History (for a specific company)
4. Naive Bayes Prediction Model

4

What type of analysis are you looking to do?:

1.Individual Company

2.Multi-Company

2

Enter the date of interest:

2021-10-21

What type of output would you like?:

Logic will display how the model variables were created, while Clean Variables will not:

 1.Logic

 2.Clean Variables

1


Model is sleeping so that it can reset

Stock Count: 92

There are 15 companies reporting earings on the specified date. Analysis will only be provided for the first 15 companies in the list. This should take about 9 minutes. Please wait patiently.


There are a total of 45 stock calls

This is the ticker:

AAL

2021-10-21

AAL


AAL has reported earnings 103 times, as of the current da

,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage
0,2021-10-21,None,-1.04,None,None
1,2021-07-22,-1.69,-1.7296,0.0396,2.2895
2,2021-04-22,-4.32,-4.3315,0.0115,0.2655
3,2021-01-28,-3.86,-4.1212,0.2612,6.338
4,2020-10-22,-5.54,-5.7547,0.2147,3.7309
5,2020-07-23,-7.82,-7.8956,0.0756,0.9575
6,2020-04-30,-2.65,-2.3551,-0.2949,-12.5218
7,2020-01-23,1.15,1.1559,-0.0059,-0.5104
8,2019-10-24,1.42,1.3975,0.0225,1.61
9,2019-07-25,1.82,1.7883,0.0317,1.7726



2. Insert the historical earnings report into the historicalEarningsTransformer function:

This is AAL's most updated earnings table that contains the earnings_event and surprise_event variables that will be used in the analysis:



,column_of_interest,reportedEPS,estimatedEPS,surprise,surprisePercentage,earnings_event,surprise_event
1,2021-07-22,-1.69,-1.7296,0.0396,2.2895,1,1
2,2021-04-22,-4.32,-4.3315,0.0115,0.2655,0,1
3,2021-01-28,-3.86,-4.1212,0.2612,6.3380,1,1
4,2020-10-22,-5.54,-5.7547,0.2147,3.7309,1,1
5,2020-07-23,-7.82,-7.8956,0.0756,0.9575,0,1
6,2020-04-30,-2.65,-2.3551,-0.2949,-12.5218,0,0
7,2020-01-23,1.15,1.1559,-0.0059,-0.5104,0,0
8,2019-10-24,1.42,1.3975,0.0225,1.6100,0,1
9,2019-07-25,1.82,1.7883,0.0317,1.7726,1,1
10,2019-04-26,0.52,0.5000,0.02,4.0000,0,1



3. Retrieve the last 5,000 daily adjusted prices for AAL in order to create the intraday_event and adjusted_price_event variables:



...

,1. open,2. high,3. low,4. close,spy_adjusted_close,6. volume,7. dividend amount,8. split coefficient,spy_intraday_returns,spy_weekly_median_return,column_of_interest,spy_event
0,451.77,453.8300,451.31,453.59,453.59,40616798,0.0,1.0,0.4029,0.3831,2021-10-21,1
1,451.13,452.7320,451.01,452.41,452.41,49571569,0.0,1.0,0.2837,0.3831,2021-10-20,0
2,448.92,450.7100,448.27,450.64,450.64,46996827,0.0,1.0,0.3831,0.2518,2021-10-19,1
3,443.97,447.5500,443.27,447.19,447.19,62213228,0.0,1.0,0.7253,0.1081,2021-10-18,1
4,444.75,446.2600,444.09,445.87,445.87,66260210,0.0,1.0,0.2518,-0.3686,2021-10-15,1
5,439.08,442.6600,438.58,442.50,442.50,70236825,0.0,1.0,0.7789,-0.3686,2021-10-14,1
6,434.71,436.0500,431.54,435.18,435.18,72973979,0.0,1.0,0.1081,-0.3686,2021-10-13,1
7,435.67,436.1000,432.78,433.62,433.62,71181163,0.0,1.0,-0.4705,0.0616,2021-10-12,0
8,437.16,440.2600,434.62,434.69,434.69,65233285,0.0,1.0,-0.5650,0.0616,2021-10-11,0
9,439.48,439.8900,437.19,437.86,437.86,74557404,0.0,1.0,-0.3686,0.6647,2021-10-08,0



5. Connect both daily adjusted price dataframes (Stock Prices and SPY Prices) with the historical earnings report in order to calculate the x variables intraday_event, adjusted_price_event, spy_event:



,1. open,2. high,3. low,4. close,stock_adjusted_close,6. volume,7. dividend amount,8. split coefficient,stock_intraday_returns,stock_weekly_median_return,column_of_interest,intraday_event,adjusted_price_event,spy_intraday_returns,spy_weekly_median_return,spy_adjusted_close,spy_event
0,27.140,27.1400,25.610,26.50,26.50,68472,0.0,1.0,-2.3581,0.1503,2021-10-21,0,1,0.4029,0.3831,453.590000,1
65,22.060,22.5676,21.990,22.06,22.06,62653,0.0,1.0,0.0000,0.7260,2021-07-21,0,1,0.1870,-0.5868,433.148580,1
123,17.900,18.1100,17.610,17.94,17.94,15348,0.0,1.0,0.2235,-1.2672,2021-04-28,1,1,-0.7494,-0.6115,414.678813,0
184,14.370,15.2432,14.020,14.26,14.26,40345,0.0,1.0,-0.7655,0.1385,2021-01-29,0,1,-2.4425,-0.8136,366.455126,0
246,9.010,10.6100,9.010,10.32,10.32,22339,0.0,1.0,14.5394,-3.5714,2020-10-29,1,0,-0.4728,-1.8457,325.364249,1
314,13.930,13.9300,13.080,13.08,13.08,12460,0.0,1.0,-6.1019,1.0638,2020-07-24,0,0,-1.8180,-1.8454,315.115236,1
375,12.740,13.6600,12.460,13.60,13.60,35604,0.0,1.0,6.7504,0.1610,2020-04-28,1,0,-4.0066,-1.9659,279.360093,0
436,20.160,20.5500,19.620,20.45,20.45,71311,0.0,1.0,1.4385,-0.3036,2020-01-30,1,1,-1.8317,-2.6849,318.418779,1
499,19.540,20.5000,19.400,19.74,19.74,64050,0.0,1.0,1.0235,-0.0519,2019-10-29,1,1,-3.2326,-3.1847,293.205112,0
565,17.950,18.2300,17.760,17.98,17.98,79440,0.0,1.0,0.1671,0.3411,2019-07-26,0,1,-3.3461,-3.6524,290.696267,1



6. Run the final tranformer function to generate the class variable:




KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Int64Index([-1], dtype='int64'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

### 5.2. Naive Bayes Prediction Model (Multi-Stock | Future Date | Clean Display)

In [23]:
if __name__=="__main__":
    main()        

#alphavantage API key
#TQ6TR98HVUPE3L9Z

#not as clean as i want it to be 

#finhub API key
#c5d5iiqad3i9ue38pn9g
#not complete

What is your alphavantage API Key?:

TQ6TR98HVUPE3L9Z

What kind of data are you looking for?

Type the number in the input box:

1. Upcoming Earnings Calendar (3 month horizon)
2. Historical Earnings (for a specific company)
3. Daily Adjusted Price History (for a specific company)
4. Naive Bayes Prediction Model

4

What type of analysis are you looking to do?:

1.Individual Company

2.Multi-Company

2

Enter the date of interest:

2021-10-21

What type of output would you like?:

Logic will display how the model variables were created, while Clean Variables will not:

 1.Logic

 2.Clean Variables

2


2021-10-21

This should be earnings calendar


,symbol,name,reportDate,estimate
6,AAL,American Airlines Group Inc,2021-10-21,-1.04
193,ALK,Alaska Air Group Inc,2021-10-21,1.29
196,ALLE,Allegion plc,2021-10-21,1.3
201,ALLY,Ally Financial Inc,2021-10-21,1.95
267,AMTB,Amerant Bancorp Inc - Class A,2021-10-21,0.39
...,...,...,...,...
4743,WAL,Western Alliance Bancorp,2021-10-21,2.22
4752,WBS,Webster Financial Corp,2021-10-21,1.08
4789,WHR,Whirlpool Corp,2021-10-21,6.12
4836,WRB,W.R. Berkley Corp,2021-10-21,0.98



Model is sleeping so that it can reset
['AAL', 'ALK', 'ALLE', 'ALLY', 'AMTB', 'AN', 'ASB', 'BANC', 'BCBP', 'BHLB', 'BJRI', 'BKU', 'BOOM', 'BSVN', 'BX', 'CE', 'CMG', 'CRI', 'CROX', 'CSL', 'CSTR', 'CZFS', 'DGX', 'DHR', 'DOW', 'EEFT', 'ESQ', 'EWBC', 'FAF', 'FCX', 'FFBC', 'GATX', 'GBCI', 'GBX', 'GPC', 'GRC', 'GTLS', 'HFWA', 'HOMB', 'HRI', 'IIIN', 'INDB', 'INTC', 'IPG', 'IQV', 'KEY', 'LNN', 'LUV', 'MAT', 'MCBS', 'MMC', 'MYFW', 'NUE', 'OLN', 'OZK', 'PBCT', 'PINE', 'POOL', 'PPBI', 'PSMT', 'PSTV', 'RHI', 'SAFE', 'SAM', 'SASR', 'SCHN', 'SCKT', 'SGLB', 'SIVB', 'SNA', 'SNAP', 'SON', 'SRCE', 'STBA', 'T', 'TMP', 'TPH', 'TRN', 'TRST', 'TRTN', 'TSCO', 'UBSI', 'UNP', 'USX', 'VICR', 'VLO', 'WABC', 'WAL', 'WBS', 'WHR', 'WRB', 'WSFS']

Stock Count: 92

There are 92 companies reporting earings on the specified date. Analysis will only be provided for the first 30 companies in the list. This should take about 12 minutes. Please wait patiently.


There are a total of 15 stock calls

This is the stock_api_c

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,Intraday_Returns,Weekly_Median_Return,trading_date
0,451.77,453.8300,451.31,453.59,453.59,40090908,0.0,1.0,0.4029,0.3831,2021-10-21
1,451.13,452.7320,451.01,452.41,452.41,49571569,0.0,1.0,0.2837,0.3831,2021-10-20
2,448.92,450.7100,448.27,450.64,450.64,46996827,0.0,1.0,0.3831,0.2518,2021-10-19
3,443.97,447.5500,443.27,447.19,447.19,62213228,0.0,1.0,0.7253,0.1081,2021-10-18
4,444.75,446.2600,444.09,445.87,445.87,66260210,0.0,1.0,0.2518,-0.3686,2021-10-15
5,439.08,442.6600,438.58,442.50,442.50,70236825,0.0,1.0,0.7789,-0.3686,2021-10-14
6,434.71,436.0500,431.54,435.18,435.18,72973979,0.0,1.0,0.1081,-0.3686,2021-10-13
7,435.67,436.1000,432.78,433.62,433.62,71181163,0.0,1.0,-0.4705,0.0616,2021-10-12
8,437.16,440.2600,434.62,434.69,434.69,65233285,0.0,1.0,-0.5650,0.0616,2021-10-11
9,439.48,439.8900,437.19,437.86,437.86,74557404,0.0,1.0,-0.3686,0.6647,2021-10-08


AAL


This is the historical earnings dataframe


...

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,Intraday_Returns,Weekly_Median_Return,trading_date
0,451.77,453.8300,451.31,453.59,453.59,40196252,0.0,1.0,0.4029,0.3831,2021-10-21
1,451.13,452.7320,451.01,452.41,452.41,49571569,0.0,1.0,0.2837,0.3831,2021-10-20
2,448.92,450.7100,448.27,450.64,450.64,46996827,0.0,1.0,0.3831,0.2518,2021-10-19
3,443.97,447.5500,443.27,447.19,447.19,62213228,0.0,1.0,0.7253,0.1081,2021-10-18
4,444.75,446.2600,444.09,445.87,445.87,66260210,0.0,1.0,0.2518,-0.3686,2021-10-15
5,439.08,442.6600,438.58,442.50,442.50,70236825,0.0,1.0,0.7789,-0.3686,2021-10-14
6,434.71,436.0500,431.54,435.18,435.18,72973979,0.0,1.0,0.1081,-0.3686,2021-10-13
7,435.67,436.1000,432.78,433.62,433.62,71181163,0.0,1.0,-0.4705,0.0616,2021-10-12
8,437.16,440.2600,434.62,434.69,434.69,65233285,0.0,1.0,-0.5650,0.0616,2021-10-11
9,439.48,439.8900,437.19,437.86,437.86,74557404,0.0,1.0,-0.3686,0.6647,2021-10-08


AMTB


This is the historical earnings dataframe


,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage
0,2021-10-21,0.45,0.39,0.06,15.3846
1,2021-07-21,0.46,0.3767,0.0833,22.1131
2,2021-04-28,0.39,0.2375,0.1525,64.2105
3,2021-01-29,0.27,0.13,0.14,107.6923
4,2020-10-29,0.08,0.056,0.024,42.8571
5,2020-07-24,-0.34,-0.24,-0.1,-41.6667
6,2020-04-28,0.09,0.165,-0.075,-45.4545
7,2020-01-30,0.26,0.22,0.04,18.1818
8,2019-10-29,0.3,0.3,0,0
9,2019-07-26,0.35,0.308,0.042,13.6364


AMTB has reported earnings 13 times, as of the current date.

This is the daily adjusted dataframe
AMTB
2021-10-21

model has been put to sleep after the daily adjusted call


Run the analysis with just clean variables


KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Int64Index([-1], dtype='int64'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [ ]:
import finnhub
finnhub_client = finnhub.Client(api_key="c5d5iiqad3i9ue38pn9g")

finub_earn_calendar = finnhub_client.earnings_calendar(_from="2021-10-21", to="2021-10-21", symbol="", international=False)
finub_earn_calendar 

### 5.3. Naive Bayes Prediction Model (Multi-Stock | Current Date | Logical Display)

In [79]:
if __name__=="__main__":
    main()        

#alphavantage API key
#TQ6TR98HVUPE3L9Z

#finhub API key
#c5d5iiqad3i9ue38pn9g

#the present date and time is 10/21/2021 at 5 PM
#HRI reported earnings today and now that its after market close, the model will make a prediction for tomorrow

What is your alphavantage API Key?:

TQ6TR98HVUPE3L9Z

What kind of data are you looking for?

Type the number in the input box:

1. Upcoming Earnings Calendar (3 month horizon)
2. Historical Earnings (for a specific company)
3. Daily Adjusted Price History (for a specific company)
4. Naive Bayes Prediction Model

4

What type of analysis are you looking to do?:

1.Individual Company

2.Multi-Company

2

Enter the date of interest:

2021-10-21

What type of output would you like?:

Logic will display how the model variables were created, while Clean Variables will not:

 1.Logic

 2.Clean Variables

1


Enter your finhub API-Key:

c5d5iiqad3i9ue38pn9g
Below is the finhub function


,symbol,reportedDate,hour,reportedEPS,estimatedEPS,surprise,surprisePercentage
0,PINE,2021-10-21,amc,0.37,0.010200,0.359800,3527.4510
1,GPC,2021-10-21,bmo,1.88,1.651350,0.228650,13.8462
2,HRI,2021-10-21,bmo,2.38,2.383600,-0.003600,-0.1510
3,IPG,2021-10-21,bmo,0.63,0.492971,0.137029,27.7966
4,KEY,2021-10-21,bmo,0.65,0.574458,0.075542,13.1502
...,...,...,...,...,...,...,...
94,SCHN,2021-10-21,bmo,1.81,1.689803,0.120197,7.1131
95,SIVB,2021-10-21,amc,6.24,5.142840,1.097160,21.3337
96,POOL,2021-10-21,bmo,4.54,3.922624,0.617376,15.7388
97,STRT,2021-10-21,amc,0.03,0.714000,-0.684000,-95.7983



Stock Count: 83

There are 83 companies reporting earings on the specified date. Analysis will only be provided for the first 15 companies in the list. This should take about 9 minutes. Please wait patiently.


There are a total of 45 stock calls

This is the ticker:

PINE

2021-10-21



This stock does not have enough data to conduct a proper naive bayes analysis.

This is the ticker:

GPC

2021-10-21



GPC has reported earnings 103 times, as of the current date.


model has been put to sleep after the daily adjusted call


1. Retrieve the historical earnings report for the stock:

This dataframe contains GPC's historical earnings report. The first row pertains to either the most recent earnings date (if the date is in the past/the same as today) or the upcoming earnings report date (if the date is in the future)



,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage
0,2021-10-21,None,1.63,None,None
1,2021-07-22,1.74,1.5421,0.1979,12.8331
2,2021-04-22,1.5,1.1439,0.3561,31.1303
3,2021-02-17,1.52,1.3455,0.1745,12.9692
4,2020-10-22,1.63,1.4249,0.2051,14.394
5,2020-07-30,1.32,0.9177,0.4023,43.8379
6,2020-05-06,0.92,1.0883,-0.1683,-15.4645
7,2020-02-19,1.35,1.303,0.047,3.6071
8,2019-10-17,1.5,1.4677,0.0323,2.2007
9,2019-07-18,1.57,1.6469,-0.0769,-4.6694



2. Insert the historical earnings report into the historicalEarningsTransformer function:

This is GPC's most updated earnings table that contains the earnings_event and surprise_event variables that will be used in the analysis:



...


6. Run the final tranformer function to generate the class variable:


Run the analysis with just clean variables

inner merged function


,column_of_interest,reportedEPS,estimatedEPS,surprise,surprisePercentage,earnings_event,surprise_event,1. open,2. high,3. low,...,7. dividend amount,8. split coefficient,stock_intraday_returns,stock_weekly_median_return,intraday_event,adjusted_price_event,spy_intraday_returns,spy_weekly_median_return,spy_adjusted_close,spy_event
0,2021-10-21,0.37,0.010200,0.3598,3527.4510,0,1,129.50,135.5494,129.09,...,0.0,1.0,3.8687,0.2610,1,1,0.4029,0.3831,453.590000,1
1,2021-10-21,1.88,1.651350,0.22865,13.8462,0,1,129.50,135.5494,129.09,...,0.0,1.0,3.8687,0.2610,1,1,0.4029,0.3831,453.590000,1
2,2021-10-21,2.38,2.383600,-0.0036,-0.1510,1,0,129.50,135.5494,129.09,...,0.0,1.0,3.8687,0.2610,1,1,0.4029,0.3831,453.590000,1
3,2021-10-21,0.63,0.492971,0.137029,27.7966,0,1,129.50,135.5494,129.09,...,0.0,1.0,3.8687,0.2610,1,1,0.4029,0.3831,453.590000,1
4,2021-10-21,0.65,0.574458,0.0755423,13.1502,1,1,129.50,135.5494,129.09,...,0.0,1.0,3.8687,0.2610,1,1,0.4029,0.3831,453.590000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,2001-02-20,0.61,0.610000,0,0.0000,1,0,27.05,28.1300,27.05,...,0.0,1.0,-46.9730,-48.4093,1,1,-33.8663,-32.8015,86.661558,0
166,2000-10-18,0.53,0.540000,-0.01,-1.8519,0,0,19.19,19.5000,18.62,...,0.0,1.0,-49.7631,-49.7584,0,0,-31.8882,-33.2474,90.333274,1
167,2000-07-19,0.55,0.550000,0,0.0000,1,0,21.31,21.3400,20.88,...,0.0,1.0,-51.2352,-50.2309,0,0,-33.2921,-32.9554,99.707461,0
168,2000-04-18,0.52,0.510000,0.01,1.9608,0,1,24.56,24.7500,24.12,...,0.0,1.0,-50.4337,-50.9341,1,1,-31.1836,-34.5057,96.730019,1



master daily adjsuted


,1. open,2. high,3. low,4. close,after_report_adjusted_price,6. volume,7. dividend amount,8. split coefficient,Intraday_Returns,Weekly_Median_Return,column_of_interest
63,127.31,129.575,126.77,129.09,128.236763,648307,0.0,1.0,0.7280,-0.9920,2021-07-23
126,120.71,122.940,119.52,122.33,120.759452,756328,0.0,1.0,0.0410,-1.5148,2021-04-23
171,98.24,101.660,98.24,101.08,99.032099,945347,0.0,1.0,0.8063,-2.9857,2021-02-18
250,97.63,99.180,97.10,98.78,95.998941,914350,0.0,1.0,-1.6707,-3.2182,2020-10-23
309,90.69,91.700,89.37,90.15,86.902974,1644819,0.0,1.0,-4.1758,-3.3838,2020-07-31
...,...,...,...,...,...,...,...,...,...,...,...
5200,27.86,28.450,27.86,28.10,14.364242,455400,0.0,1.0,-48.4413,-48.3609,2001-02-21
5284,19.31,20.000,19.06,19.88,10.023660,265400,0.0,1.0,-48.0908,-49.7631,2000-10-19
5348,20.81,21.060,20.69,20.81,10.356947,301800,0.0,1.0,-50.2309,-50.2309,2000-07-20
5411,24.56,25.000,24.25,24.69,12.143978,342100,0.0,1.0,-50.5538,-50.9341,2000-04-19


ValueError: Length of values (88) does not match length of index (170)

### 5.4. Naive Bayes Prediction Model (Multi-Stock | Current Date | Clean Display)